In [1]:
from exps.math_rlvr import RLVRMathDataset, to_datum
import rich

/Users/dipakkrishnan/git/clones/tuning/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from tinker_cookbook import tokenizer_utils

tokenizer = tokenizer_utils.get_tokenizer("Qwen/Qwen3-1.7B")

In [3]:
dataset = RLVRMathDataset(tokenizer)

##  Sample item

In [4]:
item = dataset[0]

In [5]:
rich.print(item["problem_text"])

Question: Find the domain of the expression $\frac{\sqrt{x-2}}{\sqrt{5-x}}$.}
Answer:The expressions inside each square root must be non-negative.
Therefore, $x-2 \ge 0$, so $x\ge2$, and $5 - x \ge 0$, so $x \le 5$.
Also, the denominator cannot be equal to zero, so $5-x>0$, which gives $x<5$.
Therefore, the domain of the expression is $\boxed{[2,5)}$.

Question: If $\det \mathbf{A} = 2$ and $\det \mathbf{B} = 12,$ then find $\det (\mathbf{A} \mathbf{B}).$
Answer:We have that $\det (\mathbf{A} \mathbf{B}) = (\det \mathbf{A})(\det \mathbf{B}) = (2)(12) = \boxed{24}.$

Question: Terrell usually lifts two 20-pound weights 12 times. If he uses two 15-pound weights instead, how many 
times must Terrell lift them in order to lift the same total weight?
Answer:If Terrell lifts two 20-pound weights 12 times, he lifts a total of $2\cdot 12\cdot20=480$ pounds of weight.
If he lifts two 15-pound weights instead for $n$ times, he will lift a total of $2\cdot15\cdot n=30n$ pounds of 
weight.  Equating this to 480 pounds, we can solve for $n$: \begin{align*}
30n&=480\\
\Rightarrow\qquad n&=480/30=\boxed{16}
\end{align*}

Question: If the system of equations

\begin{align*}
6x-4y&=a,\\
6y-9x &=b.
\end{align*}has a solution $(x, y)$ where $x$ and $y$ are both nonzero, find $\frac{a}{b},$ assuming $b$ is 
nonzero.
Answer:If we multiply the first equation by $-\frac{3}{2}$, we obtain

$$6y-9x=-\frac{3}{2}a.$$Since we also know that $6y-9x=b$, we have

$$-\frac{3}{2}a=b\Rightarrow\frac{a}{b}=\boxed{-\frac{2}{3}}.$$

Question: What is the modulo $13$ residue of $247+5 \cdot 39 + 7 \cdot 143 +4 \cdot 15?$

## Messages Format

This format for each question is typical of how you prompt models in this RLVR environment. You give examples in context and then have the model mimic the format of those examples. We'll compute reward as a composite of format reward and correctness.

In [6]:
rich.print(f"Ground truth: {item['ground_truth']}")
rich.print(f"Prompt tokens: {len(item['prompt_tokens'])}")

Ground truth: 8

Prompt tokens: 601

## Inference on Sample Question

Let's see what Qwen-1.7B can do on this question.

In [7]:
from exps.math_rlvr.utils import create_clients
from exps.math_rlvr.models import TrainingConfig

In [ ]:
sampling_client, training_client = create_clients(
    config=TrainingConfig()
)
